$$\textrm{Joaquin Peñuela Parra}$$
$$\textrm{Universidad de los Andes}$$
$$\textrm{Grupo de Física de Altas Energías: Fenomenología de Partículas}$$

$\textbf{Preliminaries}$ 

The libraries used here are:

In [1]:
import pandas as pd
import seaborn as sns 
import numpy as np
from scipy.interpolate import griddata

import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "serif"

This jupyter notebook defines python functions that facilitate the analysis of heatmaps associated with DataFrames stored in Excel files.

$\textbf{Functions}$

In [2]:
def read_excel(path, index_colum = 0):
    '''This function receives a path to an excel file and returns the DataFrame contained.
    
    Parameters:
    - path*: excel file path (file ending in .xlsx).
    - index_colum: column of the table that contains the indexes of the DataFrame (by default it is column 0).
    
    Return:
    - data: Pandas DataFrame.
    '''

    data = pd.read_excel(path, index_col = index_colum)
    data.sort_index(level=0, ascending=False, inplace=True)
    data.columns = [float(i) for i in data.columns]
    data.index = [float(i) for i in data.index]  
    
    return data

In [3]:
def smooth(Data, log = False):
    
    '''This function receives an array of data and interpolates the data to return a larger and "continuous" DataFrame.
        
    Parameters:
    - Data*: DataFrame with the data, each entry refers to the value of a variable based on the column and row values. For example, if we consider two independent variables A, B and a function F(A,B), Data is a matrix where the rows take the values of A and the columns the values of B, i.e. each entry in the matrix will be just F(A,B).    
    - log: Boolean that says if we will work with logarithmic scale (base 10).

    Return:
    - Data_interpolada: Pandas DataFrame.
    '''
    
    index = Data.index #Etiquetas de las columnas del DataFrame
    columns = Data.columns #Etiquetas de las filas del DataFrame
        
    if log: #Si el usuario ingresa log = True, entonces la etiqueta de la barra debe decir log y también debe sacarse el log a los datos.
        Data = np.log10(Data)
    
    #Para poder aplicar griddata (función de scipy que permite interpolar) es necesario reescribir los datos de una matriz en un formato de tres columnas. En lugar de tener una matriz donde, evaluar la función segun corresponda la fila y columna, es mejor reescribir todo en un formato de tres columnas. Simplemente: A, B, F(A,B). Esto es lo que hace el siguiente código:
    
    matriz = np.zeros([len(columns)*len(index),3])
    columna_0 = []
    for i in range(len(columns)):
        for j in range(len(index)):
            columna_0.append(columns[i])

    columna_1 = []
    for j in range(len(columns)):
        for i in range(len(index)):
            columna_1.append(index[i])

    matriz[:,0] = columna_0
    matriz[:,1] = columna_1

    for k in range(len(matriz[:,2])):
        matriz[k,2] = Data[matriz[k,0]][matriz[k,1]]

    Data = pd.DataFrame(matriz) #Se redefine los datos usando la estructura anteriormente explicada
    Data.columns = ["A", "B", "F(A,B)"] #Formato de tres columnas, se puede pensar como si fueran dos variables independientes A, B y una función F(A,B)
    
    #En este punto los datos ya se pueden leer facilmente, ahora lo que se debe hacer es crear los valores de x que queremos interpolar, es decir, los intervalos continuos que queremos interpolar, estos los llamaré "x" y "y". Sin embargo, debe tenerse en cuenta el caso en que una variable esta fija y solo la otra toma distintos valores, esto es lo que representan los len(--) == 1:
    
    if(len(columns) == 1):
        x = columns #x tiene un valor fijo
    else:
        x = np.linspace(np.min(Data['A']),np.max(Data['A']),500) #x toma un rango de valores
    
    if(len(index) == 1):
        y = index #y tiene un valor fijo
    else:
        y = np.linspace(np.min(Data['B']),np.max(Data['B']),500) #y tiene un rango de valores
    
    #En este punto ya se tienen "x" y "y", ahora debemos combinarlos para tener en cuenta todas las combinaciones, esto lo hace np.meshgrid:    
    gridx, gridy = np.meshgrid(x,y)
    
    #Finalmente bastaría interpolar usando como base nuestros datos; no obstante, hay que tener en cuenta tres casos
    
    if (len(columns) == 1):
        Data_interpolada = griddata(Data['B'].values, Data['F(A,B)'].values, y, method='cubic') #interpolación con x fijo
        Data_interpolada = pd.DataFrame(Data_interpolada)           
        Data_interpolada.index = y
        Data_interpolada.columns = x
        Data_interpolada.columns = [round(i,2) for i in Data_interpolada.columns]
        Data_interpolada.index = [round(i,2) for i in Data_interpolada.index] 
        
    elif (len(index) == 1):
        Data_interpolada = griddata(Data['A'].values, Data['F(A,B)'].values, x , method='cubic') #interpolación con y fijo
        Data_interpolada = pd.DataFrame(Data_interpolada)
        Data_interpolada.index = x
        Data_interpolada.columns = y
        Data_interpolada = Data_interpolada.T #griddata siempre retorna en forma de vector columna, pero si en este caso y esta fijo, nuestros datos deben estar en formato de vector fila, eso hace el .T, saca la transpuesta.
        Data_interpolada.columns = [round(i,2) for i in Data_interpolada.columns]
        Data_interpolada.index = [round(i,2) for i in Data_interpolada.index] 
        
    else:
        Data_interpolada = griddata((Data['A'].values,Data['B'].values), Data['F(A,B)'].values, (gridx,gridy), method='cubic')
        Data_interpolada = pd.DataFrame(Data_interpolada)
        Data_interpolada.index = y
        Data_interpolada.columns = x
            
    Data_interpolada.sort_index(level=0, ascending=False, inplace=True) #Permite que en la grafica se vea el eje y creciendo hacia arriba
    
    return Data_interpolada

In [4]:
def plot_heatmap(Data, curvas_de_nivel = {}, zoom = {}, PDF_name = '', color = 'viridis',
            titulos = {'titulo': 'Titulo Grafica','x_label': 'eje x', 'y_label': 'eje y', 'cbar_label': 'barra de color'}, curves_labels_locations = []):

    '''This function plots the heat map of a DataFrame. In addition to this, it also plots contour lines and zooms if the user wants it.
    
    Parameters:
    - curvas_de_nivel: Directory containing the level curves to be ploted. It must have the structure {valor_curva (float): letrero de la curva (string),...}.
    - titulos: Directory that contains the graph titles, xlabel, ylabel, etc.
    - cbar_label: Title of color bar (the name of the quantity that corresponds to each entry in the matrix). 
    - curves_labels_locations: List with suggested coordinates [(x1,y1), (x2,y2),...] of the positions where you want the signs of each level curve to be: The order does not matter.
    - PDF_name = Name of the file with if you to save the figure (must include the format like .pdf or .png).
    
    Return:
    - fig, ax: matplotlib.pyplot subplots.
    '''

    fig, ax = plt.subplots()
    plt.title(titulos['titulo'])
    
    try: plt.title(titulos['titulo_derecha'], loc = 'right')
    except: pass

    try: plt.title(titulos['titulo_izquierda'], loc = 'left')
    except: pass

    index = Data.index
    columns = Data.columns
    
    if(len(index) == 1 or len(columns) == 1):
        Data.sort_index(level=0, ascending=False, inplace=True) #Permite que en la grafica se vea el eje y creciendo hacia arriba
        sns.heatmap(Data, cmap = color, cbar_kws={'label': titulos['cbar_label']}).set(xlabel= titulos['x_label'], ylabel= titulos['y_label'])
        
    else:
        mapa_calor = plt.pcolormesh(columns, index, Data.values, cmap = color)
        plt.colorbar(mapa_calor, label = titulos['cbar_label'])  
    
        plt.xlabel(titulos['x_label'])
        plt.ylabel(titulos['y_label'])
    
    #Dependiendo la forma de los datos las curvas se hacen diferente, en el caso en que una de las dos variables independientes este fija entonces no se puede usar la función contour de matplotlib por la dimensión de dicha variable, en ese caso es necesario graficar asintotas.

    if (len(list(curvas_de_nivel.keys())) != 0):

        index = Data.index
        columns = Data.columns

        if(len(index) == 1):
            linestyles = ['-', '--', '-.', ':', '']
            for i in range(len(list(curvas_de_nivel.keys()))):
                curva = list(curvas_de_nivel.keys())[i]
                indice = np.abs(np.asarray(Data - curva)).argmin()
                
                #Se grafica con el letrero dentro de la grafica:
                ax.axvline(indice, ls = linestyles[1], color = 'white') #Asintota para cada curva con y fijo
                ax.text(indice - 20, 0.5, curvas_de_nivel[curva], rotation = 90, color = 'white')
                
                #Se grafica con el letrero en un legend
                #ax.axvline(indice, label = curvas_de_nivel[curva], ls = linestyles[i % 5], color = 'white') #Asintota para cada curva con y fijo
            #ax.legend()

        elif(len(columns) == 1):
            
            linestyles = ['-', '--', '-.', ':', '']
            for i in range(len(list(curvas_de_nivel.keys()))):
                curva = list(curvas_de_nivel.keys())[i]
                indice = np.abs(np.asarray(Data - curva)).argmin()
                
                #Se grafica con el letrero dentro de la grafica:
                ax.axhline(indice, ls = linestyles[1], color = 'white') #Asintota para cada curva con x fijo
                ax.text(0.5, indice + 20 , curvas_de_nivel[curva], color = 'white')
                
                #Se grafica con el letrero en un legend
                #ax.axhline(indice, label = curvas_de_nivel[curva], ls = linestyles[i % 5], color = 'white') #Asintota para cada curva con x fijo
            #ax.legend()
            
        else:
            curves = plt.contour(Data.columns, Data.index, Data.values, levels = list(sorted(curvas_de_nivel.keys())), colors = ['white'], linestyles = 'dashed') #Curvas de nivel
            if (len(curves_labels_locations) != 0): ax.clabel(curves, curves.levels, manual = curves_labels_locations, fmt = curvas_de_nivel, fontsize=10, rightside_up = True) #Letreros de las curvas de nivel
            else: ax.clabel(curves, curves.levels, inline=True, fmt = curvas_de_nivel, fontsize=10, rightside_up = True) #Letreros de las curvas de nivel

    if (len(list(zoom.keys())) != 0):
        
        if (len(index) == 1 or len(columns) == 1): print('No es posible hacer zoom por las dimensiones del DataFrame')
        
        else:
        
            ax_zoom = ax.inset_axes([1.4, 0, 1, 1])
            
            ax_zoom.pcolormesh(Data.columns, Data.index, Data.values, cmap = color)
            
            if (len(list(curvas_de_nivel.keys())) != 0): ax_zoom.contour(Data.columns, Data.index, Data.values, levels = list(sorted(curvas_de_nivel.keys())), colors = ['white'], linestyles = 'dashed') #Curvas de nivel

            ax_zoom.set_xlim(zoom['x1'], zoom['x2'])
            ax_zoom.set_ylim(zoom['y1'], zoom['y2'])
            ax_zoom.set_xlabel(titulos['x_label'])
            ax_zoom.set_ylabel(titulos['y_label'])

            ax.indicate_inset_zoom(ax_zoom, edgecolor= "black")
            
    if (PDF_name != ''): plt.savefig(PDF_name, bbox_inches='tight')
    
    return fig, ax